In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('nlp').getOrCreate()

In [3]:
from pyspark.ml.feature import HashingTF,IDF,Tokenizer

In [4]:
sentenceData=spark.createDataFrame([
  (0.0,"hi I heard about spark"),
  (0.0,"I wish java could use case classes"),
  (1.0,"logistic regression model are neat")
],['label','sentence'])

In [5]:
sentenceData.show()

+-----+--------------------+
label| sentence|
+-----+--------------------+
 0.0|hi I heard about ...|
 0.0|I wish java could...|
 1.0|logistic regressi...|
+-----+--------------------+

In [6]:
tokenizer=Tokenizer(inputCol='sentence',outputCol='words')

In [7]:
words_data=tokenizer.transform(sentenceData)

In [8]:
words_data.show()

+-----+--------------------+--------------------+
label| sentence| words|
+-----+--------------------+--------------------+
 0.0|hi I heard about ...|[hi, i, heard, ab...|
 0.0|I wish java could...|[i, wish, java, c...|
 1.0|logistic regressi...|[logistic, regres...|
+-----+--------------------+--------------------+

In [9]:
words_data.show(truncate=False)

+-----+----------------------------------+------------------------------------------+
label|sentence |words |
+-----+----------------------------------+------------------------------------------+
0.0 |hi I heard about spark |[hi, i, heard, about, spark] |
0.0 |I wish java could use case classes|[i, wish, java, could, use, case, classes]|
1.0 |logistic regression model are neat|[logistic, regression, model, are, neat] |
+-----+----------------------------------+------------------------------------------+

In [10]:
hashing_tf=HashingTF(inputCol='words',outputCol='rawFeatures')

In [11]:
featurized_data=hashing_tf.transform(words_data)

In [12]:
idf=IDF(inputCol='rawFeatures',outputCol='features')

In [13]:
idf_model=idf.fit(featurized_data)

In [14]:
rescaled_data=idf_model.transform(featurized_data)

In [15]:
rescaled_data.select('label','features').show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(262144,[24417,49...|
 0.0|(262144,[20719,24...|
 1.0|(262144,[13671,91...|
+-----+--------------------+

In [16]:
from pyspark.ml.feature import CountVectorizer

In [17]:
df=spark.createDataFrame([
  (0,"a b c".split(" ")),
  (1,"a b b c a".split(" "))
],["id","words"])

In [18]:
df.show()

+---+---------------+
 id| words|
+---+---------------+
 0| [a, b, c]|
 1|[a, b, b, c, a]|
+---+---------------+

In [19]:
cv=CountVectorizer(inputCol='words',outputCol='features',vocabSize=3,minDF=2.0)

In [20]:
model=cv.fit(df)

In [21]:
result=model.transform(df)

In [22]:
result.show(truncate=False)

+---+---------------+-------------------------+
id |words |features |
+---+---------------+-------------------------+
0 |[a, b, c] |(3,[0,1,2],[1.0,1.0,1.0])|
1 |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+